<a href="https://colab.research.google.com/github/ai-fast-track/icevision-gradio/blob/master/IceApp_pets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install icedata

     |████████████████████████████████| 2.8MB 2.7MB/s 
     |████████████████████████████████| 112kB 32.8MB/s 
     |████████████████████████████████| 122kB 23.8MB/s 
     |████████████████████████████████| 952kB 20.4MB/s 
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=65165 sha256=9dfa204f96af77cf241a52b85dd2d20e165ddcef9badde1b0e956bf447cca194
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [2]:
pip install gradio

     |████████████████████████████████| 1.0MB 2.7MB/s 
     |████████████████████████████████| 215kB 10.4MB/s 
     |████████████████████████████████| 2.6MB 15.2MB/s 
     |████████████████████████████████| 962kB 34.3MB/s 
     |████████████████████████████████| 71kB 8.8MB/s 


In [3]:
from icevision.all import *
import icedata
import PIL, requests
import torch
from torchvision import transforms
import gradio as gr

In [4]:
class_map = icedata.pets.class_map()
model = icedata.pets.trained_models.faster_rcnn_resnet50_fpn()

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


Downloading: "https://github.com/airctic/model_zoo/releases/download/pets_faster_resnet50fpn/pets_faster_resnetfpn50.zip" to /root/.cache/torch/hub/checkpoints/pets_faster_resnetfpn50.zip


In [13]:
def predict(
    model, image, detection_threshold: float = 0.5, mask_threshold: float = 0.5
):
    tfms_ = tfms.A.Adapter([tfms.A.Normalize()])
    # Whenever you have images in memory (numpy arrays) you can use `Dataset.from_images`
    infer_ds = Dataset.from_images([image], tfms_)

    batch, samples = faster_rcnn.build_infer_batch(infer_ds)
    preds = faster_rcnn.predict(
        model=model,
        batch=batch,
        detection_threshold=detection_threshold
    )
    return samples[0]["img"], preds[0]

def show_preds(input_image, display_list, detection_threshold):
    display_label = ("Label" in display_list)
    display_bbox = ("BBox" in display_list)

    if detection_threshold==0: detection_threshold=0.5
    
    img, pred = predict(model=model, image=input_image, detection_threshold=detection_threshold)
    # print(pred)
    img = draw_pred(img=img, pred=pred, class_map=class_map, denormalize_fn=denormalize_imagenet, display_label=display_label, display_bbox=display_bbox)
    img = PIL.Image.fromarray(img)
    # print("Output Image: ", img.size, type(img))
    return img

In [ ]:
display_chkbox = gr.inputs.CheckboxGroup(["Label", "BBox"], label="Display")
detection_threshold_slider = gr.inputs.Slider(minimum=0, maximum=1, step=0.1, default=0.5, label="Detection Threshold")

outputs = gr.outputs.Image(type="pil")

gr_interface = gr.Interface(fn=show_preds, inputs=["image", display_chkbox,  detection_threshold_slider], outputs=outputs, title='IceApp - PETS')
gr_interface.launch(inline=False, share=True, debug=True)
